# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
import os
import sys
from dotenv import load_dotenv
from pathlib import Path

project_root = Path().resolve()
sys.path.append(str(project_root / 'starter'))

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from tavily import TavilyClient
import chromadb

In [2]:
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
print(f".env file found at: {env_path}")

load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
CHROMA_OPENAI_API_KEY=os.getenv("CHROMA_OPENAI_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

print("OPENAI_API_KEY:", OPENAI_API_KEY[:10] + "..." if OPENAI_API_KEY else "None")  # Masked

.env file found at: /Users/ahsan/Documents/Udemy/ai-research-agent/.env
OPENAI_API_KEY: voc-122867...


### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [3]:
chroma_client = chromadb.PersistentClient(path="./starter/chromadb")
collection = chroma_client.get_collection("udaplay")

@tool
def retrieve_game(query: str) -> str:
    """
    Semantic search: Finds most results in the vector DB
    args:
        - query: a question about game industry. 

    Returns results as list. Each element contains:
        - Platform: like Game Boy, Playstation 5, Xbox 360...)
        - Name: Name of the Game
        - YearOfRelease: Year when that game was released for that platform
        - Description: Additional details about the game
    """
    results = collection.query(
        query_texts=[query],
        n_results=5,
    )

    games = []

    for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
        games.append({
            "Platform": meta.get("Platform"),
            "Name": meta.get("Name"),
            "YearOfRelease": meta.get("YearOfRelease"),
            "Description": doc
        })

    return games

#### Evaluate Retrieval Tool

In [4]:
@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> dict:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    llm = LLM(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", base_url="https://openai.vocareum.com/v1")

    prompt = f"""
    Your task is to evaluate if the documents are enough to respond the query.
    Question: {question}
    Retrieved Documents: {retrieved_docs}
    Provide a detailed explanation, so it's possible to take an action to accept it or not.
    """

    response = llm.invoke([
        SystemMessage(content="You are an expert evaluator."),
        UserMessage(content=prompt)
    ])

    useful = "yes" in response.content.lower()
    return {
        "useful": useful,
        "description": response.content
    }

#### Game Web Search Tool

In [5]:
@tool
def game_web_search(question: str) -> str:
    """
    Web search: Finds most relevant web results about game industry
    args:
        - question: a question about game industry
    return:
      top 5 search results.
    """
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    search_results = tavily_client.search(query=question, num_results=5)
    return search_results

### Agent

In [6]:
agent = Agent(
    model_name="gpt-4o-mini",
    temperature=0,
    api_key=OPENAI_API_KEY,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    instructions="""
    You are Udaplay, an AI research agent specialized in the video game industry.
    
    Your goal is to provide accurate and comprehensive answers to user queries about video games.
    You have access to the following tools:
    1. retrieve_game: Use this tool to perform semantic searches in the video game database.
    2. evaluate_retrieval: Use this tool to evaluate the usefulness of retrieved documents.
    3. game_web_search: Use this tool to search the web for additional information about video games.
    
    Always ensure that your responses are well-supported by the information you gather using these tools.
    """
)

In [7]:
responses = []
questions = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5?"
]

for question in questions:
    try:
        print('--------------------------------\n')
        print(f"Question: {question}")
        response = agent.invoke(question)
        final_state = response.get_final_state()
        final_message = final_state["messages"][-1] 
        print(f"LLM Response: {final_message.content}\n")
        responses.append(response)
        print('/n')
        print('--------------------------------\n')
    except Exception as e:
        print(f"Error processing question: {e}\n")
        responses.append(None)

--------------------------------

Question: When Pokémon Gold and Silver was released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
LLM Response: Pokémon Gold and Silver were released for the Game Boy Color in 1999. These games are part of the second generation of Pokémon and introduced new regions, Pokémon, and gameplay mechanics.

/n
--------------------------------

--------------------------------

Question: Which one was the first 3D platformer Mario game?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
LLM Response: Pokémon Gold and Silver were released for the Gam

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes

In [7]:
from lib.tooling import Tool
from lib.memory import LongTermMemory, MemoryFragment
from lib.vector_db import VectorStoreManager


In [8]:
def build_memory_register_tool(ltm: LongTermMemory, owner: str, namespace: str):
   def _register(content: str) -> str:
       """
       Registers a new memory entry into the long-term memory.
       args:
           - content: The content to be stored in memory.
       returns:
           - confirmation message.
       """
       ltm.register(MemoryFragment(
           content=content,
           namespace=namespace,
           owner=owner
       ))
       return "Memory registered successfully."
   
   return Tool(
       func=_register,
       name='register_memory',
       description="""
       Register new memory or anything you have learned which you did not know earlier about any games. 
       
       Args: content to save
       """
   )

In [9]:
def search_memory_tool(ltm: LongTermMemory, owner: str, namespace: str):
    def _search(content: str):
        result = ltm.search(
            query_text=content,
            owner=owner,
            namespace=namespace,
            limit=2
        )
    return Tool(
       func=_search,
       name='search_memory',
       description="""
       Search long term memory, before utilising the other tools
       
       Args: Information to look for
       """
   )

In [10]:
db = VectorStoreManager(OPENAI_API_KEY)
vector_store = db.get_or_create_store('ltm_store_new')
ltm = LongTermMemory(db)

In [11]:
agent_ltm = Agent(
    model_name="gpt-4o-mini",
    temperature=0,
    api_key=OPENAI_API_KEY,
    tools=[retrieve_game, 
           evaluate_retrieval, 
           game_web_search, 
           build_memory_register_tool(ltm, 'GameStore', "Facts"),
           search_memory_tool(ltm, 'GameStore', "Facts")
           ],
    instructions="""
    You are Udaplay, an AI research agent specialized in the video game industry.
        
        Your goal is to provide accurate and comprehensive answers to user queries about video games.

        Instructions:
        - Try to use memory, over all other tools
        - If no results are found use other tooling.
        - If you use web search tool, save this content to long term memory.

        You have access to the following tools:
        1. build_memory_register_tool: Use this tool to save content to long term memory
        2. search_memory_tool: Use this tool to search long term memory
        3. retrieve_game: Use this tool to perform semantic searches in the video game database.
        4. evaluate_retrieval: Use this tool to evaluate the usefulness of retrieved documents.
        5. game_web_search: Use this tool to search the web for additional information about video games.

        
        Always ensure that your responses are well-supported by the information you gather using these tools.
    """
)


In [13]:
new_question = "When is PS8 being released?"

try:
    print('--------------------------------\n')
    print(f"Question: {new_question}")
    response = agent_ltm.invoke(new_question)
    final_state = response.get_final_state()
    final_message = final_state["messages"][-1] 
    print(f"LLM Response: {final_message.content}\n")
    print('/n')
    print('--------------------------------\n')
except Exception as e:
    print(f"Error processing question: {e}\n")
    responses.append(None)

--------------------------------

Question: When is PS8 being released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
LLM Response: The PlayStation 8 is expected to be released around 2040 or 2042, based on the typical 6 to 7-year gap between new console releases by Sony. 

For more information, you can refer to the following sources:
- [Quora discussion on PlayStation 8 release](https://www.quora.com/When-will-the-PlayStation-8-come-out)

/n
--------------------------------

[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
LLM Response: The PlayStation 8 is expected to be released around 2040 or 2042, based on the typical 6 to 7-year gap between new

In [15]:
reask_question = "When is PS8 being released?"
try:
    print('--------------------------------\n')
    print(f"Question: {reask_question}")
    response = agent_ltm.invoke(reask_question)
    final_state = response.get_final_state()
    final_message = final_state["messages"][-1] 
    print(f"LLM Response: {final_message.content}\n")
    print('/n')
    print('--------------------------------\n')
except Exception as e:
    print(f"Error processing question: {e}\n")
    responses.append(None)

--------------------------------

Question: When is PS8 being released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
LLM Response: The PlayStation 8 is anticipated to be released around 2040 or 2042. This estimate is based on Sony's historical pattern of releasing new consoles approximately every 6 to 7 years.

For further details, you can refer to the following sources:
- [Quora discussion on PlayStation 8 release](https://www.quora.com/When-will-the-PlayStation-8-come-out)

/n
--------------------------------
